In [1]:
import os 
from PIL import Image, ImageDraw
import numpy as np
import tqdm
from tensorflow.keras.models import load_model

In [2]:
loaded_model = load_model('face_rec_acc_plus.h5')

2024-04-17 16:17:53.817057: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-17 16:17:53.817079: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-17 16:17:53.817084: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-17 16:17:53.817382: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-17 16:17:53.817406: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
def image_gen():
    output_dir = "generated_images"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        image_size=(160,160)
        num_images = 70000
        for i in range(num_images):
            # Create a blank image with a white background
            image = Image.new("RGB", image_size, "white")
            
            # Create a drawing object
            draw = ImageDraw.Draw(image)
            
            # Randomize parameters for shape generation
            shape = np.random.choice(['rectangle', 'circle', 'triangle'])
            color = tuple(np.random.randint(0, 256, size=3))
            position = tuple(np.random.randint(0, size - 50) for size in image_size)
            size = np.random.randint(20, 80)
            
            # Draw a shape on the image
            if shape == 'rectangle':
                end_position = (position[0] + size, position[1] + size)
                draw.rectangle([position, end_position], fill=color)
            elif shape == 'circle':
                end_position = (position[0] + size, position[1] + size)
                draw.ellipse([position, end_position], fill=color)
            elif shape == 'triangle':
                points = [
                    (position[0] + size // 2, position[1]),
                    (position[0], position[1] + size),
                    (position[0] + size, position[1] + size)
                ]
                draw.polygon(points, fill=color)
            
            # Save the image
            image_path = os.path.join(output_dir, f"image_{i}.png")
            image.save(image_path)

            if i % 1000 == 0:
                print(f"Generated {i}/{num_images} images")

        print("Image generation completed!")


In [ ]:
# create image
image_gen()

In [5]:
import sys
sys.stdout = open(os.devnull, 'w')

#preprocess image and ransform to numpy array
non_defined_features = []
for i in tqdm.tqdm(range(70000), desc="Preprocessing images"):
    image = Image.open(f"generated_images/image_{i}.png")
    image = image.resize((160, 160))
    image = np.array(image) / 255.0
    image = image.reshape(1, 160, 160, 3)
    non_defined_features.append(loaded_model.predict(image)[0])
sys.stdout = sys.__stdout__
          

Preprocessing images: 100%|██████████| 70000/70000 [48:40<00:00, 23.96it/s]  


In [6]:
non_defined_features=np.array(non_defined_features)

AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [7]:
non_defined_features

array([[3.65893720e-05, 4.32615423e-07, 4.18030359e-05, ...,
        3.13520991e-06, 8.63364221e-07, 2.38137794e-08],
       [1.24119222e-03, 9.71614213e-07, 1.19273714e-03, ...,
        3.81360201e-06, 1.60367508e-05, 9.78692150e-09],
       [7.62555937e-05, 1.00013565e-07, 6.56283464e-06, ...,
        1.03698558e-05, 2.63486783e-07, 4.48002098e-08],
       ...,
       [5.04511081e-06, 1.53209356e-07, 2.24752621e-05, ...,
        1.10354517e-06, 1.30401190e-06, 8.05205413e-09],
       [8.66095506e-05, 8.45677022e-08, 1.71757220e-05, ...,
        1.92938260e-05, 7.73360512e-07, 8.08986442e-08],
       [2.15022847e-05, 3.13334738e-07, 2.17363577e-05, ...,
        7.40853147e-06, 1.23898042e-06, 3.36366099e-08]], dtype=float32)

In [8]:
np.save('non_defined_features.npy', non_defined_features)
